<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/typy-nowszych-danych/Typy_nowszych_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aktualny setup sparka

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 57 kB/s 
     |████████████████████████████████| 200 kB 84.4 MB/s 
     |████████████████████████████████| 199 kB 21.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=d0c960021c53d561503ff4a95462bb078962333b43fcd511a9b4858e65228808
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2022-12-14 20:33:57--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop2.tgz’

spark-3.3.1-bin-had 100%[===============

In [2]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import split, col, monotonically_increasing_id, when
from pyspark.sql.types import StructType, StringType, IntegerType, BooleanType, FloatType, TimestampType, DateType, ArrayType, MapType
from typing import List, Tuple, Dict, Any
import numpy
from google.colab import drive

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"

In [4]:
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

In [5]:
! pip install kaggle
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


##Import danych

In [6]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-12-14 20:34:09--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.39.96, 13.249.39.59, 13.249.39.82, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.39.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237820761 (227M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 226.80M  80.3MB/s    in 2.8s    

2022-12-14 20:34:12 (80.3 MB/s) - ‘name.basics.tsv.gz’ saved [237820761/237820761]

--2022-12-14 20:34:12--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.39.96, 13.249.39.59, 13.249.39.82, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.39.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290768499 (277M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 277.30M  34.5M

In [7]:
! kaggle datasets download unanimad/the-oscar-award
! kaggle datasets download unanimad/golden-globe-awards
! kaggle datasets download unanimad/emmy-awards
! kaggle datasets download tmdb/tmdb-movie-metadata

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. O

##Rozpakowanie danych

In [8]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basics.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

In [9]:
! unzip the-oscar-award.zip
! unzip golden-globe-awards.zip
! unzip emmy-awards.zip 
! unzip tmdb-movie-metadata.zip

unzip:  cannot find or open the-oscar-award.zip, the-oscar-award.zip.zip or the-oscar-award.zip.ZIP.
unzip:  cannot find or open golden-globe-awards.zip, golden-globe-awards.zip.zip or golden-globe-awards.zip.ZIP.
unzip:  cannot find or open emmy-awards.zip, emmy-awards.zip.zip or emmy-awards.zip.ZIP.
unzip:  cannot find or open tmdb-movie-metadata.zip, tmdb-movie-metadata.zip.zip or tmdb-movie-metadata.zip.ZIP.


#Wczytanie danych

##Ustalenie typów danych

In [10]:
map_types = {
    str : StringType(),
    int : IntegerType(),
    bool : BooleanType(),
    float: FloatType(),
    'timestamp' : TimestampType(),
    'date' : DateType(),
    List[str] : ArrayType(StringType()),
    Tuple[str] : ArrayType(StringType()),
    Dict[str, str] : MapType(StringType(), StringType())
}

In [11]:
column_conf = {
  'akas'  :  ['titleId','ordering','title','region','language','types','attributes','isOriginalTitle'], 
  'title_basics' : ['tconst','titleType','primaryTitle','originalTitle','isAdult','startYear','endYear','runtimeMinutes','genres'],
  'crew' : ['tconst','directors','writers'],
  'episode' : ['tconst','parentTconst','seasonNumber','episodeNumber'],
  'principals' : ['tconst','ordering','nconst','category','job','characters'],
  'ratings' : ['tconst','averageRating','numVotes'],
  'name_basics' : ['nconst','primaryName','birthYear','deathYear','primaryProfession','knownForTitles'],
  'oscars': ['year_film', 'year_ceremony', 'ceremony', 'category', 'name', 'film', 'winner'],
  'globe':['year_film', 'year_award', 'ceremony', 'category', 'nominee', 'film', 'win'],
  'emmy_awards_category': ['name', 'category', 'role'],
  'emmy_awards': ['id', 'year', 'category', 'nominee', 'staff', 'company', 'producer', 'win'],
  'tmdb_credits': ['movie_id', 'title', 'cast', 'crew'],
  'tmdb_movies':['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview',\
                 'popularity', 'production_companies','production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', \
                 'status', 'tagline', 'title', 'vote_average', 'vote_count']
}
column_type_collection = {
    int : ['ordering', 'startYear', 'endYear', 'runtimeMinutes', 'seasonNumber', 'episodeNumber', 'numVotes', 'birthYear', 'deathYear', \
           'year_film', 'year_award', 'ceremony', 'year_film', 'year_ceremony', 'ceremony', 'year', 'movie_id', 'budget', \
            'isOriginalTitle', 'isAdult'],
    str : ['titleId', 'title', 'region', 'language', 'types', 'attributes', 'tconst', 'titleType', 'primaryTitle', 'originalTitle', 'genres', \
           'directors', 'writers', 'parentTconst', 'category', 'job', 'characters', 'primaryName', 'primaryProfession', 'knownForTitles', \
           'category', 'nominee', 'film', 'category', 'name', 'film', 'role', 'original_language', 'original_title', 'overview', 'staff', \
           'company', 'producer', 'title', 'cast', 'crew', 'genres', 'keywords', 'production_companies', 'production_countries', 'homepage',\
           'spoken_languages', 'status', 'tagline', 'id', 'popularity', 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count'],
    bool : ['win', 'winner'],
    float : ['averageRating'],
    'date': []
}

In [12]:
def init_schema(conf, column_type_collection):
  map = {}
  for pole in conf:
    for python_type, column_list in column_type_collection.items():
      if pole in column_list:
        map[pole] = map_types[python_type]
  schemat= StructType()
  for pole, typ in map.items():
    schemat = schemat.add(pole, typ, True)
  return schemat

In [13]:
Schematy=[schemat_title_akas, schemat_title_basics, schemat_title_crew, schemat_title_episode, schemat_title_principals,
schemat_title_ratings, schemat_name_basics, schemat_oscars, schemat_globe, schemat_emmy_awards_category, schemat_emmy_awards, schemat_tmdb_credits, schemat_tmdb_movies] = [ 
init_schema(column_conf[table], column_type_collection) 
  for table in ('akas', 'title_basics', 'crew', 'episode', 'principals', 'ratings', 'name_basics', 'oscars', 'globe', 'emmy_awards_category', 'emmy_awards', 'tmdb_credits', 'tmdb_movies')]

##Wczytajmy dane

In [14]:
CSV=['title.akas.csv', 'title.basics.csv', 'title.crew.csv','title.episode.csv', 'title.principals.csv',
'title.ratings.csv', 'name.basics.csv', 'the_oscar_award.csv', 'golden_globe_awards.csv', 'emmy_awards_categories.csv', 'the_emmy_awards.csv', 'tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']

In [15]:
def upload(schemat, csv):
  df=spark.read.option("header","true").option("delimiter", "\t").schema(schemat).csv(csv)
  return df
  
def upload2(schemat, csv):
  df=spark.read.option("header","true").schema(schemat).csv(csv)
  return df

In [16]:
[df_title_akas, df_title_basics, df_title_crew, df_title_episode, df_title_principals,
df_title_ratings, df_name_basics]=[
upload(Schematy[i], CSV[i])
  for i in range(7) 
]

In [17]:
[oscars, globe, emmy_awards_category, emmy_awards, tmdb_credits, tmdb_movies]=[
upload2(Schematy[7+i], CSV[7+i])
  for i in range(6) 
]

#Zamiana "\\N" na null

In [18]:
for column in ['primaryProfession', 'knownForTitles']:
  df_name_basics = df_name_basics.withColumn(column, when(df_name_basics[column] == "\\N", None).otherwise(df_name_basics[column]))

for column in ['title', 'region', 'language', 'types', 'attributes']:
  df_title_akas = df_title_akas.withColumn(column, when(df_title_akas[column] == "\\N", None).otherwise(df_title_akas[column]))

for column in ['titleType', 'primaryTitle', 'originalTitle', 'genres']:
  df_title_basics = df_title_basics.withColumn(column, when(df_title_basics[column] == "\\N", None).otherwise(df_title_basics[column]))

for column in ['directors', 'writers']:
  df_title_crew = df_title_crew.withColumn(column, when(df_title_crew[column] == "\\N", None).otherwise(df_title_crew[column]))

df_title_episode=df_title_episode.withColumn('parentTconst', when(df_title_episode['parentTconst'] == "\\N", None).otherwise(df_title_episode['parentTconst']))

for column in ['ordering', 'category', 'job', 'characters']:
  df_title_principals = df_title_principals.withColumn(column, when(df_title_principals[column] == "\\N", None).otherwise(df_title_principals[column]))

#Zamiana string na array(string) i "0"/"1" na bool

In [19]:
df_title_akas = df_title_akas.withColumn("isOriginalTitle", df_title_akas["isOriginalTitle"].cast(BooleanType()))
df_title_basics = df_title_basics.withColumn("isAdult", df_title_basics["isAdult"].cast(BooleanType()))

Zmienimy na array następujące kolumny: z tabeli df_title_akas kolumny attributes, types;
z tabeli df_title_basics kolumna genres;
z tabeli df_name_basics kolumny primaryProfession, knownForTitles;
z tabeli df_title_crew kolumny directors, writers.


##Stworzenie nowych, odrębnych kolumn typu array, z kolumn typu string określonych wyżej

In [20]:
df_title_akas_types= df_title_akas.select(split(col("types"),",").alias("types"))
df_title_akas_attributes= df_title_akas.select(split(col("attributes"),",").alias("attributes"))
df_name_basics_primaryProfession= df_name_basics.select(split(col("primaryProfession"),",").alias("primaryProfession"))
df_name_basics_knownForTitles= df_name_basics.select(split(col("knownForTitles"),",").alias("knownForTitles"))
df_title_basics_genres=df_title_basics.select(split(col("genres"),",").alias("genres"))
df_title_crew_directors=df_title_crew.select(split(col("directors"),",").alias("directors"))
df_title_crew_writers=df_title_crew.select(split(col("writers"),",").alias("writers"))

In [21]:
emmy_awards_staff=emmy_awards.select(split(col("staff"),";").alias("staff"))
emmy_awards_producer=emmy_awards.select(split(col("producer"),";").alias("producer"))

##Zamieńmy pierwotne kolumny na nowe

###usuńmy pierwotne kolumny z niewłaściwym typem danych

In [22]:
df_title_akas= df_title_akas.drop("types")
df_title_akas= df_title_akas.drop("attributes")
df_name_basics= df_name_basics.drop("primaryProfession")
df_name_basics= df_name_basics.drop("knownForTitles")
df_title_basics=df_title_basics.drop("genres")
df_title_crew=df_title_crew.drop("directors")
df_title_crew=df_title_crew.drop("writers")

In [23]:
emmy_awards=emmy_awards.drop("staff")
emmy_awards=emmy_awards.drop("producer")

###dodajmy kolumny ideksów do nowych (jednokolumnowych) tabel i tabel pierwotnych

dodajmy kolumny indeksów do bazowych tabel

In [24]:
df_title_akas=df_title_akas.select("*").withColumn("id", monotonically_increasing_id())
df_name_basics=df_name_basics.select("*").withColumn("id", monotonically_increasing_id())
df_title_basics=df_title_basics.select("*").withColumn("id", monotonically_increasing_id())
df_title_crew= df_title_crew.select("*").withColumn("id", monotonically_increasing_id())

In [25]:
emmy_awards=emmy_awards.select("*").withColumn("id", monotonically_increasing_id())

i do nowych, jednokolumnowych tabel

In [26]:
df_title_akas_types=df_title_akas_types.select("*").withColumn("id1", monotonically_increasing_id())
df_title_akas_attributes=df_title_akas_attributes.select("*").withColumn("id2", monotonically_increasing_id())
df_name_basics_primaryProfession=df_name_basics_primaryProfession.select("*").withColumn("id3", monotonically_increasing_id())
df_name_basics_knownForTitles=df_name_basics_knownForTitles.select("*").withColumn("id4", monotonically_increasing_id())
df_title_basics_genres=df_title_basics_genres.select("*").withColumn("id5", monotonically_increasing_id())
df_title_crew_directors=df_title_crew_directors.select("*").withColumn("id6", monotonically_increasing_id())
df_title_crew_writers=df_title_crew_writers.select("*").withColumn("id7", monotonically_increasing_id())

In [27]:
emmy_awards_staff=emmy_awards_staff.select("*").withColumn("id8", monotonically_increasing_id())
emmy_awards_producer=emmy_awards_producer.select("*").withColumn("id9", monotonically_increasing_id())

###połączmy je odpowiednio na bazie numerów indeksu

In [28]:
df_title_akas= df_title_akas.join(df_title_akas_types, col('id') == col('id1'), 'leftouter')
df_title_akas= df_title_akas.join(df_title_akas_attributes, col('id') == col('id2'), 'leftouter')
df_name_basics= df_name_basics.join(df_name_basics_primaryProfession, col('id') == col('id3'), 'leftouter')
df_name_basics= df_name_basics.join(df_name_basics_knownForTitles, col('id') == col('id4'), 'leftouter')
df_title_basics=df_title_basics.join(df_title_basics_genres, col('id') == col('id5'), 'leftouter')
df_title_crew= df_title_crew.join(df_title_crew_directors, col('id') == col('id6'), 'leftouter')
df_title_crew= df_title_crew.join(df_title_crew_writers, col('id') == col('id7'), 'leftouter')

In [29]:
emmy_awards=emmy_awards.join(emmy_awards_staff, col('id') == col('id8'), 'leftouter')
emmy_awards=emmy_awards.join(emmy_awards_producer, col('id') == col('id9'), 'leftouter')

###usuńmy kolumny indeksów

In [30]:
df_title_akas=df_title_akas.drop("id").drop("id1").drop("id2")
df_name_basics=df_name_basics.drop("id").drop("id3").drop("id4")
df_title_basics=df_title_basics.drop("id").drop("id5")
df_title_crew=df_title_crew.drop("id").drop("id6").drop("id7")

In [31]:
emmy_awards=emmy_awards.drop("id").drop("id8").drop("id9")

#Usunięcie duplikatów

In [32]:
Data=[df_title_akas, df_title_basics, df_title_crew, df_title_episode, df_title_principals, \
           df_title_ratings, df_name_basics, oscars, globe, emmy_awards_category, emmy_awards, tmdb_credits, tmdb_movies]
for df in Data:
  df=df.distinct()

#Wyświetlmy dane

In [33]:
for df in Data:
  df.show(3)

+---------+--------+---------------+------+--------+---------------+-------------+----------+
|  titleId|ordering|          title|region|language|isOriginalTitle|        types|attributes|
+---------+--------+---------------+------+--------+---------------+-------------+----------+
|tt0000001|       1|     Карменсіта|    UA|    null|          false|[imdbDisplay]|      null|
|tt0670946|       6|Épisode #1.1149|    FR|      fr|          false|         null|      null|
|tt1334523|       4|  Heart of Time|   XWW|      en|          false|         null|      null|
+---------+--------+---------------+------+--------+---------------+-------------+----------+
only showing top 3 rows

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+----------+---------+--------------------+--------------------

In [34]:
for df in Data:
  df.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- isOriginalTitle: boolean (nullable = true)
 |-- types: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- attributes: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: boolean (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- endYear: integer (nullable = true)
 |-- runtimeMinutes: integer (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- tconst: string (nullable = true)
 |-- directors: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- writers: a